## NOAO data reduction
### WESmith

MIT License

Copyright (c) 2018 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
import os
import fnmatch
import numpy as np
import pandas as pd

In [2]:
pd.set_option('max_rows', 32, 'max_columns', 40)

In [3]:
# some important fields: 
important = ['DATE-OBS', 'DTCALDAT', 'DTTELESC', 'DTINSTRU',
             'OBSTYPE','PROCTYPE','PRODTYPE','DTSITE', 'OBSERVAT', 
             'REFERENCE','FILESIZE','MD5SUM','DTACQNAM','DTPROPID',
             'PI','RELEASE_DATE','RA','DEC','FOOTPRINT','FILTER',
             'EXPOSURE','OBSMODE','SEEING','DEPTH','SURVEYID',
             'COLLECTIONID','OBJECT','RADIUS / BOX', 'RADIUS/BOX']  # note RADIUS/BOX with and without spaces

In [4]:
# location of test NOAO json data
BASE = '/Users/smithw/python/noao_data/json-scrape/mtn'
# the following will depend upon the organization of NOAO data
DATE = ['{}'.format(x) for x in range(20170701, 20170726)] 

In [5]:
# HDF5 storage of dataframe metadata from 
# https://stackoverflow.com/questions/29129095/save-additional-attributes-in-pandas-dataframe/29130146#29130146
# note: needed to 'pip install --upgrade tables' for HDFStore

def h5store(filename, df, **kwargs):
    store = pd.HDFStore(filename)
    store.put('mydata', df)
    store.get_storer('mydata').attrs.metadata = kwargs
    store.close()

def h5load(store):
    data = store['mydata']
    metadata = store.get_storer('mydata').attrs.metadata
    return data, metadata

In [60]:
#%%writefile ProcessJSON.txt
# to write this cell out for printing, uncomment the line above: 
# otherwise leave it commented, or this cell will not compile

class ProcessJSON(object):

    def __init__(self, datadir, savdir='../pydata-book/processed', file_hdr='local_file'):
        self._datadir   = datadir
        self._savdir    = savdir
        self._file_hdr  = file_hdr
        self._txtfmt    = 'DATE:{}, {} FILES'  # date, number of files that date
        self._savfmt    = '{}/{}-processed.hdf5' # savdir, date
        self._errmsg    = 'ERROR: Need to run .run() method first!'
        self._error_group_col = \
              'ERROR: grouping column {} in file {} does not have a unique value'
            
        self._metadata        = {}
        self._date            = None
        self._important       = None  # no self-importance here!
        self._processed       = None
        self._group_col       = None
        self._num             = None
        self._num_to_read     = None
        self._full_dataframe  = None
        self._multi_group_cols = None
        self._force_overwrite  = False
        
        os.makedirs(self._savdir, exist_ok=True)
        
    def _get_file_list(self):
        file_list = []
        for dirpath, dirs, files in os.walk(os.path.join(self._datadir, self._date)): 
            for filename in fnmatch.filter(files, '*.json'):
                file_list.append(os.path.join(dirpath, filename))
        self._num = min(len(file_list), self._num_to_read)
        self._file_list = file_list
               
    def _process(self):
        '''process group of json files , save dataframe to disk'''
        
        self._get_file_list()
        print('processing ', self._txtfmt.format(self._date, self._num))
        
        # if important keys are provided, make a dummy starting dataframe with those keys
        dd = [] if self._important == None else [pd.DataFrame(columns=self._important)]
        
        for k in range(self._num):
            jj = pd.read_json(self._file_list[k])
            
            # verify the grouping-column value is unique and not missing
            # in this file across the HDUs, otherwise assert an error; 
            # TODO: make this a try/except: save bad filenames and keep moving
            assert jj[self._group_col].nunique() == 1, \
                self._error_group_col.format(self._group_col, self._file_list[k])
            
            # if existing and unique, broadcast the grouping-column value
            # to the entire grouping column: this is required for proper grouping later;
            # usually grouping column is 'DTINSTRU', the instrument name
            jj[self._group_col] = jj[self._group_col].dropna().iloc[0]
            
            # add the file-name column to the dataframe: 
            # this is required for grouping HDUs by filename
            jj[self._file_hdr] = self._file_list[k][47:]
            
            dd.append(jj)
            
        # if important keys are provided, cull the dataframe with those keys: 
        # do this AFTER concat with dummy frame with all the important keys, 
        # otherwise smaller frames may not have all of the keys
        self._processed = pd.concat(dd) if self._important == None else \
                          pd.concat(dd)[self._important]
            
        self._metadata['num_files']   = self._num
        self._metadata['date_record'] = self._date
        h5store(self._savfile, self._processed, **self._metadata)
        
    def _get_data(self, date):
        self._date = date
        self._savfile = self._savfmt.format(self._savdir, self._date)
        if (os.path.isfile(self._savfile) and not self._force_overwrite):
            print('reading {} from disk'.format(self._savfile))
            with pd.HDFStore(self._savfile) as store:
                self._processed, self._metadata = h5load(store)
        else:
            self._process()
                    
    def run(self, date_range, group_col='DTINSTRU', important=None, 
            num_to_read=None, force_overwrite=False):  
        '''
        date_range:      list of date directories to read
        group_col:       column name on which to group: usually 'DTINSTRU'
        important:       list of columns to keep in processed dataframes
        num_to_read:     number of files to read 
                         (default: read all files in each date directory)
        force_overwrite: if processed dataframe exists on disk, 
                         overwrite if True (default: False)
        '''
        raw = []
        self._num_to_read = np.iinfo(np.int32).max if num_to_read == None else \
                            num_to_read
        self._group_col   = group_col
        self._multi_group_cols  = [self._group_col, self._file_hdr]

        # add file-header column for filename: 
        # important not to use append() method here: it breaks things
        self._important = important + [self._file_hdr] 
        self._force_overwrite = force_overwrite
        
        # ensure date_range is a list if a scalar is input
        date_range = date_range if isinstance(date_range, list) else [date_range]
        
        for k in date_range:
            self._get_data(k)
            raw.append(self._processed)
        self._full_dataframe = pd.concat(raw)
        
    @property
    def get_full_dataframe(self):
        assert self._full_dataframe is not None, self._errmsg
        return self._full_dataframe
    
    @property
    def get_instr_vs_fields_unique_all_data(self):
        # TODO: this needs to be generalized so user can define the top rows 
        #       for display: 
        #       this will be broken when the 'important' list changes
        gg = self.get_full_dataframe.groupby(self._group_col).nunique().T
        indx = list(gg.index)
        # reorder rows to get similar rows at top for direct comparison
        indx = [self._file_hdr,'DTACQNAM'] + indx[:12] + indx[13:-1]
        return gg.loc[indx,:]
    
    @property
    def get_HDU_uniqueness_per_file(self):
        gg = self.get_full_dataframe.groupby(self._multi_group_cols).nunique()
        # drop corrupted (by nunique()) grouping columns
        gg = gg.drop(self._multi_group_cols, axis=1)
        # reset index, drop unnecessary local_file column
        gg.reset_index().drop(self._multi_group_cols[1], axis=1)  
        return gg.groupby(self._multi_group_cols[0]).\
                  agg(['min','max','mean','std']).round(2).stack().T
    
    @property
    def get_all_fields(self):
        return self._important
    
    @property    
    def get_HDU_stats(self):
        gg = self.get_full_dataframe.groupby(self._multi_group_cols).size()
        return gg.groupby(self._group_col).agg(['min','max','mean','std']).\
                                         rename_axis('HDU stats:', axis=1)
    
    def get_num_files_writing_fields(self, instr=True, percent=True):
        '''
        instr:   if True, list percentages (or raw numbers) of files per 
                 instrument that write each field, if False list total
                 number of files (or percentages) over ALL instruments 
                 (default=True)
        percent: if True, list percentages of files that write each field, 
                 if False, list raw numbers of files (default=True)
        '''
        zz = self.get_full_dataframe.groupby(self._multi_group_cols).nunique() > 0
        if not instr:
            gg = zz.sum()
            return (gg/gg[self._file_hdr]*100).round(2) if percent else gg
        else:
            gg = zz.drop(['DTINSTRU'], axis=1).\
                 rename(columns={self._file_hdr:'COUNT'}).\
                 reset_index().drop(self._file_hdr, axis=1)
            gg = gg.groupby('DTINSTRU').sum().T
            return (gg/gg.loc['COUNT']*100).round(2) if percent else gg

    def get_unique_values_of_field(self, field):
        return list(self.get_full_dataframe[field].dropna().unique())
    
    def get_num_unique_values_by_keys(self, field1, field2):
        gg = self.get_full_dataframe.groupby([field1, field2]).nunique()
        return pd.DataFrame(gg.loc[:, self._file_hdr]).rename(columns=\
                                            {self._file_hdr:'TOTAL OCCURRENCES'})

In [61]:
proc = ProcessJSON(BASE)

In [62]:
dates = DATE
num = None #100  # 'None' to get all files
force_overwrite = False
proc.run(dates, important=important, group_col='DTINSTRU', num_to_read=num, force_overwrite=force_overwrite) 

reading ../pydata-book/processed/20170701-processed.hdf5 from disk
reading ../pydata-book/processed/20170702-processed.hdf5 from disk
reading ../pydata-book/processed/20170703-processed.hdf5 from disk
reading ../pydata-book/processed/20170704-processed.hdf5 from disk
reading ../pydata-book/processed/20170705-processed.hdf5 from disk
reading ../pydata-book/processed/20170706-processed.hdf5 from disk
reading ../pydata-book/processed/20170707-processed.hdf5 from disk
reading ../pydata-book/processed/20170708-processed.hdf5 from disk
reading ../pydata-book/processed/20170709-processed.hdf5 from disk
reading ../pydata-book/processed/20170710-processed.hdf5 from disk
reading ../pydata-book/processed/20170711-processed.hdf5 from disk
reading ../pydata-book/processed/20170712-processed.hdf5 from disk
reading ../pydata-book/processed/20170713-processed.hdf5 from disk
reading ../pydata-book/processed/20170714-processed.hdf5 from disk
reading ../pydata-book/processed/20170715-processed.hdf5 from 

## TESTING

In [9]:
aa = proc.get_full_dataframe.copy()  # make copy to experiment: without copying, it is a VIEW (ie, a pointer)

In [10]:
bb = proc.get_instr_vs_fields_unique_all_data

In [11]:
cc = proc.get_HDU_uniqueness_per_file

In [12]:
dd = proc.get_all_fields # list

In [13]:
ee = proc.get_HDU_stats

In [14]:
ff = proc.get_unique_values_of_field('OBSTYPE')

In [15]:
gg1 = proc.get_num_unique_values_by_keys('DTINSTRU', 'OBSTYPE')

In [16]:
gg2 = proc.get_num_unique_values_by_keys('DTINSTRU', 'FILTER')

In [17]:
gg3 = proc.get_num_unique_values_by_keys('DTTELESC','DTINSTRU')

In [18]:
gg4 = proc.get_num_unique_values_by_keys('DTINSTRU', 'DTCALDAT')

In [78]:
hh1 = proc.get_num_files_writing_fields(instr=True, percent=True)

In [79]:
hh2 = proc.get_num_files_writing_fields(instr=True, percent=False)

In [80]:
hh3 = proc.get_num_files_writing_fields(instr=False, percent=True)

In [81]:
hh4 = proc.get_num_files_writing_fields(instr=False, percent=False)

In [19]:
aa # too big for html: 389000 rows!

,DATE-OBS,DTCALDAT,DTTELESC,DTINSTRU,OBSTYPE,PROCTYPE,PRODTYPE,DTSITE,OBSERVAT,REFERENCE,FILESIZE,MD5SUM,DTACQNAM,DTPROPID,PI,RELEASE_DATE,RA,DEC,FOOTPRINT,FILTER,EXPOSURE,OBSMODE,SEEING,DEPTH,SURVEYID,COLLECTIONID,OBJECT,RADIUS / BOX,RADIUS/BOX,local_file
0,2017-07-01T21:20:01.445,2017-07-01,ct13m,andicam,BIAS,raw,image,ct,CTIO,NaN,NaN,NaN,/lhome/data/observer/ccd170701bias.0001.fits,smarts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212001_zri.f...
1,2017-07-01,NaN,NaN,andicam,NaN,NaN,NaN,NaN,CTIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:17:30.03,-30:06:56.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIASES,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212001_zri.f...
0,2017-07-01T21:20:52.614,2017-07-01,ct13m,andicam,BIAS,raw,image,ct,CTIO,NaN,NaN,NaN,/lhome/data/observer/ccd170701bias.0002.fits,smarts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212052_zri.f...
1,2017-07-01,NaN,NaN,andicam,NaN,NaN,NaN,NaN,CTIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:18:21.27,-30:06:56.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIASES,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212052_zri.f...
0,2017-07-01T21:21:43.718,2017-07-01,ct13m,andicam,BIAS,raw,image,ct,CTIO,NaN,NaN,NaN,/lhome/data/observer/ccd170701bias.0003.fits,smarts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212143_zri.f...
1,2017-07-01,NaN,NaN,andicam,NaN,NaN,NaN,NaN,CTIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:19:12.52,-30:06:56.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIASES,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212143_zri.f...
0,2017-07-01T21:22:34.807,2017-07-01,ct13m,andicam,BIAS,raw,image,ct,CTIO,NaN,NaN,NaN,/lhome/data/observer/ccd170701bias.0004.fits,smarts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212234_zri.f...
1,2017-07-01,NaN,NaN,andicam,NaN,NaN,NaN,NaN,CTIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:20:03.55,-30:06:56.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIASES,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212234_zri.f...
0,2017-07-01T21:23:25.909,2017-07-01,ct13m,andicam,BIAS,raw,image,ct,CTIO,NaN,NaN,NaN,/lhome/data/observer/ccd170701bias.0005.fits,smarts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212325_zri.f...
1,2017-07-01,NaN,NaN,andicam,NaN,NaN,NaN,NaN,CTIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:20:54.79,-30:06:56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIASES,NaN,NaN,20170701/ct13m/smarts/c13a_170701_212325_zri.f...


In [23]:
# TODO: make optional csv, html output a method in ProcessJSON
bb.to_html('html/get_instr_vs_fields_unique_all_data.html')

In [25]:
bb.to_csv('csv/get_instr_vs_fields_unique_all_data.csv')

In [26]:
cc.to_html('html/get_HDU_uniqueness_per_file.html')

In [27]:
cc.to_csv('csv/get_HDU_uniqueness_per_file.csv')

In [28]:
dd  # list

['DATE-OBS',
 'DTCALDAT',
 'DTTELESC',
 'DTINSTRU',
 'OBSTYPE',
 'PROCTYPE',
 'PRODTYPE',
 'DTSITE',
 'OBSERVAT',
 'REFERENCE',
 'FILESIZE',
 'MD5SUM',
 'DTACQNAM',
 'DTPROPID',
 'PI',
 'RELEASE_DATE',
 'RA',
 'DEC',
 'FOOTPRINT',
 'FILTER',
 'EXPOSURE',
 'OBSMODE',
 'SEEING',
 'DEPTH',
 'SURVEYID',
 'COLLECTIONID',
 'OBJECT',
 'RADIUS / BOX',
 'RADIUS/BOX',
 'local_file']

In [29]:
ee.to_html('html/get_HDU_stats.html')

In [30]:
ee.to_csv('csv/get_HDU_stats.csv')

In [31]:
gg1.to_html('html/get_num_unique_values_by_keys_DTINSTRU_OBSTYPE.html')

In [32]:
gg1.to_csv('csv/get_num_unique_values_by_keys_DTINSTRU_OBSTYPE.csv')

In [33]:
gg2.to_html('html/get_num_unique_values_by_keys_DTINSTRU_FILTER.html')

In [34]:
gg2.to_csv('csv/get_num_unique_values_by_keys_DTINSTRU_FILTER.csv')

In [35]:
gg3.to_html('html/get_num_unique_values_by_keys_DTTELESC_DTINSTRU.html')

In [36]:
gg3.to_csv('csv/get_num_unique_values_by_keys_DTTELESC_DTINSTRU.csv')

In [37]:
gg4.to_html('html/get_num_unique_values_by_keys_DTINSTRU_DTCALDAT.html')

In [38]:
gg4.to_csv('csv/get_num_unique_values_by_keys_DTINSTRU_DTCALDAT.csv')

In [70]:
hh1.to_html('html/get_num_files_writing_fields(instr=True, percent=True).html')

In [71]:
hh2.to_html('html/get_num_files_writing_fields(instr=True, percent=False).html')

In [75]:
hh3.to_frame().to_html('html/get_num_files_writing_fields(instr=False, percent=True).html')

In [77]:
hh4.to_frame().to_html('html/get_num_files_writing_fields(instr=False, percent=False).html')